<a href="https://colab.research.google.com/github/delsishah/construction360/blob/main/Optimized_Heirarchy_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="BBD2KSgO7DkV3LT4HKiE")
workspace = rf.workspace()

# Replace these IDs with your actual project IDs
excavating = workspace.project("my-first-project-0hp9o").version(2).download("folder") #rename to My-First-Project-1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.8 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-2 in folder:: 100%|██████████| 3560/3560 [00:02<00:00, 1723.44it/s]


In [2]:
!pip install -q roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="P8UG8mKYY92QP5Z517t4")
workspace = rf.workspace()

# Replace these IDs with your actual project IDs
bricklaying = workspace.project("brick-by-brick").version(3).download("folder")
concreting = workspace.project("cementing").version(1).download("folder")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Brick-by-Brick-3 in folder:: 100%|██████████| 13496/13496 [00:02<00:00, 6252.59it/s]

loading Roboflow project...



Extracting Dataset Version Zip to Cementing-1 in folder:: 100%|██████████| 2046/2046 [00:00<00:00, 3689.06it/s]


In [3]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split


In [5]:
# Main categories (mutually exclusive)
MAIN_CATEGORIES = ['brick-masonry', 'concreting', 'excavation']

# Sublabels per category (excluding Complete/Incomplete which are shared)
SUBLABELS = {
    'brick-masonry': ['brick-laying', 'capping'],
    'concreting': ['compaction', 'pouring', 'smoothing'],
    'excavation': ['site-clearing', 'digging']
}

# Shared labels
SHARED_LABELS = ['complete', 'incomplete']

def clean_labels(folder_name):
    # Lowercase and split
    labels = folder_name.lower().split()

    # Identify main category (only one)
    main_cat = None
    for mc in MAIN_CATEGORIES:
        if mc in labels:
            main_cat = mc
            break
    if main_cat is None:
        return None, None, None  # invalid folder

    # Separate sublabels (excluding main_cat and shared labels)
    sublabels = [l for l in labels if l != main_cat and l not in SHARED_LABELS]

    # Shared labels: pick complete or incomplete if present (default to incomplete)
    shared = [l for l in labels if l in SHARED_LABELS]
    shared_label = shared[0] if shared else 'incomplete'

    return main_cat, shared_label, sublabels


In [7]:
base_dirs = [
    "/content/My-First-Project-1/train",
    "/content/Cementing-1/train",
    "/content/Brick-by-Brick-3/train"
]

image_paths = []
main_categories = []
shared_labels = []
sublabels_all = []

for base_dir in base_dirs:
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        main_cat, shared_label, sublabels = clean_labels(folder)
        if main_cat is None:
            print(f"Skipping invalid folder: {folder}")
            continue

        # List all images in this folder
        imgs = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.png','.jpg','.jpeg'))]

        image_paths.extend(imgs)
        main_categories.extend([main_cat]*len(imgs))
        shared_labels.extend([shared_label]*len(imgs))
        sublabels_all.extend([sublabels]*len(imgs))

print(f"Loaded {len(image_paths)} images")


Skipping invalid folder: digging
Skipping invalid folder: Unlabeled
Skipping invalid folder: Incomplete Site-Clearing
Skipping invalid folder: Incomplete digging
Skipping invalid folder: Compaction
Skipping invalid folder: Brick-Laying
Loaded 15398 images


In [8]:
import shutil

# Replace with your actual dataset folder name
shutil.rmtree('/content/My-First-Project-2', ignore_errors=True)

In [9]:
# Encode main category (single label)
le_main = LabelEncoder()
main_cat_encoded = le_main.fit_transform(main_categories)  # e.g. brick-masonry -> 0, concreting -> 1, excavation -> 2

# Encode shared labels (complete/incomplete) - binary
le_shared = LabelEncoder()
shared_encoded = le_shared.fit_transform(shared_labels)  # complete->1, incomplete->0

# Create MultiLabelBinarizers for each subcategory
mlb_sub = {}
for cat in MAIN_CATEGORIES:
    mlb_sub[cat] = MultiLabelBinarizer(classes=SUBLABELS[cat])
    mlb_sub[cat].fit([SUBLABELS[cat]])

# Prepare sublabels encoded arrays with zeros for all, will fill accordingly
sublabels_encoded = {
    cat: np.zeros((len(image_paths), len(SUBLABELS[cat]))) for cat in MAIN_CATEGORIES
}

# Fill the appropriate entries in the sublabels_encoded arrays
for i, (main_cat, sublab) in enumerate(zip(main_categories, sublabels_all)):
    # Encode sublabels for this sample according to its main_cat
    # This gives a binary vector for the current sample's sublabels
    encoded_vector = mlb_sub[main_cat].transform([sublab])[0]

    # Place the encoded vector at the correct index 'i' in the array for the main_cat
    sublabels_encoded[main_cat][i] = encoded_vector

# Now sublabels_encoded[cat] will have shape (num_images, num_sublabels_in_cat)

In [10]:
IMG_SIZE = 224
BATCH_SIZE = 32

def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0
    return img


In [11]:
# Split indices for train/val (same for all models)
train_idx, val_idx = train_test_split(np.arange(len(image_paths)), test_size=0.2, stratify=main_cat_encoded, random_state=42)

def create_dataset(indices):
    paths = [image_paths[i] for i in indices]
    main_y = main_cat_encoded[indices]
    shared_y = shared_encoded[indices]
    sub_y = {cat: sublabels_encoded[cat][indices] for cat in MAIN_CATEGORIES}
    return paths, main_y, shared_y, sub_y

train_paths, train_main_y, train_shared_y, train_sub_y = create_dataset(train_idx)
val_paths, val_main_y, val_shared_y, val_sub_y = create_dataset(val_idx)

# Dataset generator function for main model (main category + shared label)
def main_dataset_generator(paths, main_labels, shared_labels):
    def gen():
        for p, m, s in zip(paths, main_labels, shared_labels):
            yield p, (m, s)
    return gen

def preprocess_main_sample(path, labels):
    img = load_and_preprocess_image(path)
    # main label is categorical int -> one-hot
    main_cat_onehot = tf.one_hot(labels[0], len(MAIN_CATEGORIES))
    # shared label is binary scalar
    shared_label = tf.cast(labels[1], tf.float32)
    return img, (main_cat_onehot, shared_label)

train_ds_main = tf.data.Dataset.from_generator(main_dataset_generator(train_paths, train_main_y, train_shared_y),
                                               output_signature=(
                                                   tf.TensorSpec(shape=(), dtype=tf.string),
                                                   (tf.TensorSpec(shape=(), dtype=tf.int32),
                                                    tf.TensorSpec(shape=(), dtype=tf.int32))
                                               ))
train_ds_main = train_ds_main.map(preprocess_main_sample).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds_main = tf.data.Dataset.from_generator(main_dataset_generator(val_paths, val_main_y, val_shared_y),
                                             output_signature=(
                                                 tf.TensorSpec(shape=(), dtype=tf.string),
                                                 (tf.TensorSpec(shape=(), dtype=tf.int32),
                                                  tf.TensorSpec(shape=(), dtype=tf.int32))
                                             ))
val_ds_main = val_ds_main.map(preprocess_main_sample).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [12]:
def build_main_model():
    inp = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model = EfficientNetB0(include_top=False, input_tensor=inp, weights='imagenet')
    x = GlobalAveragePooling2D()(base_model.output)
    # main category output (softmax)
    main_out = Dense(len(MAIN_CATEGORIES), activation='softmax', name='main_cat')(x)
    # shared output (sigmoid)
    shared_out = Dense(1, activation='sigmoid', name='shared')(x)

    model = Model(inputs=inp, outputs=[main_out, shared_out])
    model.compile(optimizer=Adam(1e-4),
                  loss={'main_cat': 'categorical_crossentropy', 'shared': 'binary_crossentropy'},
                  metrics={'main_cat': 'accuracy', 'shared': 'accuracy'})
    return model

main_model = build_main_model()
main_model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 4,012,672 (15.31 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [13]:
def build_sub_model(num_labels):
    inp = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model = EfficientNetB0(include_top=False, input_tensor=inp, weights='imagenet')
    x = GlobalAveragePooling2D()(base_model.output)
    out = Dense(num_labels, activation='sigmoid')(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer=Adam(1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

sub_models = {}
for cat in MAIN_CATEGORIES:
    sub_models[cat] = build_sub_model(len(SUBLABELS[cat]))
    print(f"{cat} model built.")


brick-masonry model built.
concreting model built.
excavation model built.


In [14]:
def filter_by_category(paths, sub_labels, main_labels, cat):
    # Only select samples belonging to main category cat
    indices = [i for i, m in enumerate(main_labels) if le_main.inverse_transform([m])[0] == cat]
    filtered_paths = [paths[i] for i in indices]
    filtered_sub = sub_labels[indices]
    return filtered_paths, filtered_sub

train_sub_datasets = {}
val_sub_datasets = {}

for cat in MAIN_CATEGORIES:
    tr_paths, tr_sub = filter_by_category(train_paths, train_sub_y[cat], train_main_y, cat)
    val_paths_cat, val_sub = filter_by_category(val_paths, val_sub_y[cat], val_main_y, cat)

    def sub_dataset_generator(paths, labels):
        def gen():
            for p, l in zip(paths, labels):
                yield p, l
        return gen

    def preprocess_sub_sample(path, label):
        img = load_and_preprocess_image(path)
        label = tf.cast(label, tf.float32)
        return img, label

    train_ds = tf.data.Dataset.from_generator(sub_dataset_generator(tr_paths, tr_sub),
                                              output_signature=(
                                                  tf.TensorSpec(shape=(), dtype=tf.string),
                                                  tf.TensorSpec(shape=(len(SUBLABELS[cat]),), dtype=tf.int64)
                                              ))
    train_ds = train_ds.map(preprocess_sub_sample).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    val_ds = tf.data.Dataset.from_generator(sub_dataset_generator(val_paths_cat, val_sub),
                                            output_signature=(
                                                tf.TensorSpec(shape=(), dtype=tf.string),
                                                tf.TensorSpec(shape=(len(SUBLABELS[cat]),), dtype=tf.int64)
                                            ))
    val_ds = val_ds.map(preprocess_sub_sample).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    train_sub_datasets[cat] = train_ds
    val_sub_datasets[cat] = val_ds


In [15]:
# Train Stage 1 model
main_model.fit(train_ds_main,
               validation_data=val_ds_main,
               epochs=3)

# Train Stage 2 submodels
for cat in MAIN_CATEGORIES:
    print(f"Training {cat} sub-model...")
    sub_models[cat].fit(train_sub_datasets[cat],
                        validation_data=val_sub_datasets[cat],
                        epochs=5)


Epoch 1/3
    385/Unknown 3801s 10s/step - loss: 0.4373 - main_cat_accuracy: 0.9368 - main_cat_loss: 0.2137 - shared_accuracy: 0.8990 - shared_loss: 0.2236

Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.


385/385 ━━━━━━━━━━━━━━━━━━━━ 4002s 10s/step - loss: 0.4365 - main_cat_accuracy: 0.9369 - main_cat_loss: 0.2133 - shared_accuracy: 0.8992 - shared_loss: 0.2232 - val_loss: 3.3196 - val_main_cat_accuracy: 0.1101 - val_main_cat_loss: 2.9287 - val_shared_accuracy: 0.8834 - val_shared_loss: 0.3599
Epoch 2/3
385/385 ━━━━━━━━━━━━━━━━━━━━ 3925s 10s/step - loss: 0.0153 - main_cat_accuracy: 0.9978 - main_cat_loss: 0.0078 - shared_accuracy: 0.9985 - shared_loss: 0.0076 - val_loss: 1.3016 - val_main_cat_accuracy: 0.4981 - val_main_cat_loss: 1.2723 - val_shared_accuracy: 0.9984 - val_shared_loss: 0.0156
Epoch 3/3
385/385 ━━━━━━━━━━━━━━━━━━━━ 3937s 10s/step - loss: 0.0044 - main_cat_accuracy: 0.9997 - main_cat_loss: 0.0016 - shared_accuracy: 0.9992 - shared_loss: 0.0027 - val_loss: 0.0439 - val_main_cat_accuracy: 0.9942 - val_main_cat_loss: 0.0179 - val_shared_accuracy: 0.9945 - val_shared_loss: 0.0252
Training brick-masonry sub-model...
Epoch 1/5
294/294 ━━━━━━━━━━━━━━━━━━━━ 3096s 10s/step - accura

In [17]:
main_model.save("main_model.keras")
sub_models['brick-masonry'].save("submodel_brick-masonry.keras")
sub_models['concreting'].save("submodel_concreting.keras")
sub_models['excavation'].save("submodel_excavation.keras")
